In [1]:
import numpy as np
import torch
from tsGaussian.torch_tsgaussian import TangentSpaceGaussian
from stable_baselines_utils import TangentSpaceGaussian as TSG

In [2]:
tg = TangentSpaceGaussian(None)

# Test liegroup torch

In [3]:
from liegroups.torch import SO3

In [4]:
C = SO3.exp(torch.Tensor([[1,2,3],
                         [0,0,0]]))
C

<liegroups.torch.so3.SO3Matrix>
| tensor([[[-0.6949,  0.7135,  0.0893],
|          [-0.1920, -0.3038,  0.9332],
|          [ 0.6930,  0.6313,  0.3481]],
| 
|         [[ 1.0000,  0.0000,  0.0000],
|          [ 0.0000,  1.0000,  0.0000],
|          [ 0.0000,  0.0000,  1.0000]]])

# Test torch_tsgaussian sample

In [5]:
R_mu = torch.eye(3).reshape((1,3,3))
sigma = torch.ones(3).reshape((1,3))

In [6]:
tg.rsample(R_mu, sigma)

omega size:  torch.Size([1, 3])


tensor([[[-0.8605, -0.4307, -0.2721],
         [-0.4563,  0.8891,  0.0354],
         [ 0.2267,  0.1546, -0.9616]]])

# Test torch_tsgaussian normal_term

In [7]:
sigma = torch.ones(3).reshape((1,3))
sigma

tensor([[1., 1., 1.]])

In [8]:
tg.normal_term(sigma)

tensor([15.7496])

# Test torch_tsgaussian log_map

In [9]:
R_1 = torch.eye(3).reshape((1, 3, 3))
R_2 = torch.eye(3).reshape((1, 3, 3))

In [10]:
tg.log_map(R_1, R_2)

tensor([0., 0., 0.])

# Test torch_tsgaussian log_probs

In [11]:
R_x = torch.eye(3).reshape((1,3,3))
R_mu = torch.zeros(3,3).reshape((1,3,3))
R_x = R_x.repeat(5, 1, 1)
R_mu = R_mu.repeat(5, 1, 1)
sigma = torch.ones(3).reshape((1,3))

In [12]:
tg.log_probs(R_x, R_mu, sigma)

tensor([-2.7568, -2.7568, -2.7568, -2.7568, -2.7568])

In [13]:
np.e ** (-2.7568)

0.06349462641817973

all codes run for torch_tsgaussian now, need to check it's correctness and make it into batch version.

# Test TangentSpaceGaussian actions_from_params

In [14]:
tsg = TSG(None)

In [15]:
print(tsg.distribution)

In [16]:
tsg

In [17]:
tsg.actions_from_params(torch.eye(3).reshape((1,3,3)), torch.ones(3).reshape((1,3)))

omega size:  torch.Size([1, 3])


tensor([[[ 0.4645, -0.5245,  0.7135],
         [-0.1455,  0.7496,  0.6458],
         [-0.8735, -0.4038,  0.2719]]])

# Test TangentSpaceGaussian log_prob_from_params

In [18]:
torch.eye(3).repeat(2,1,1).size()

torch.Size([2, 3, 3])

In [19]:
torch.ones(3).repeat(2,1).size()

torch.Size([2, 3])

In [20]:
tsg.log_prob_from_params(torch.eye(3).repeat(2,1,1), torch.ones(3))

omega size:  torch.Size([3])
actions:  torch.Size([2, 3, 3])
mu:  torch.Size([2, 3, 3])
sigma:  torch.Size([3])


(tensor([[[ 0.5467, -0.7767, -0.3130],
          [ 0.3601, -0.1194,  0.9253],
          [-0.7560, -0.6185,  0.2144]],
 
         [[ 0.5467, -0.7767, -0.3130],
          [ 0.3601, -0.1194,  0.9253],
          [-0.7560, -0.6185,  0.2144]]]),
 tensor([0.3089, 0.3089]))

In [21]:
x = torch.randn(2, 4, 4)
y = torch.linalg.inv(x)
y

tensor([[[ 0.2662,  0.6123, -0.3697, -0.4346],
         [ 0.1505,  0.3033,  0.1720,  0.0158],
         [-1.3446, -0.7496,  0.2459,  0.9484],
         [ 1.2190,  0.2261,  0.3008, -1.3324]],

        [[-0.0308, -2.7073,  0.5094,  0.3284],
         [ 0.5661, -3.6981,  0.3530, -0.2007],
         [-0.0381, -6.0140,  1.7558,  0.0379],
         [ 0.1416, -0.8358, -0.8445, -0.1856]]])

Again, codes can run, but need to check correctness.

# Try to run training

In [22]:
import torch
from absl import app, flags
from stable_baselines3 import SAC, PPO
from envs.wahba import Wahba
from stable_baselines_utils import CustomSACPolicy, \
    CustomCNN

In [23]:
def main(argv):
    env = Wahba()
    device = torch.device('cpu')
    policy_kwargs = dict(
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = dict(features_dim = 256))
    policy_kwargs['n_critics'] = 1
    policy_kwargs['share_features_extractor'] = False
    policy = CustomSACPolicy
    model = SAC(policy, env, verbose = 1, ent_coef = 'auto_0.1',
                policy_kwargs = policy_kwargs, device = device)
    model.learn(total_timesteps = 110, eval_freq = 5, n_eval_episodes = 5)

In [24]:
main(None)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -4.17    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 517      |
|    time_elapsed    | 0        |
|    total_timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -3.91    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 582      |
|    time_elapsed    | 0        |
|    total_timesteps | 8        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -4.51    |
| time/              |          |
|    episodes        | 12       |
|    fps             |

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.77    |
| time/              |          |
|    episodes        | 100      |
|    fps             | 806      |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------


/home/fantasticoven/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


torch.Size([1, 3, 3])
torch.Size([1, 3, 3])
omega size:  torch.Size([1, 3, 3])


/home/fantasticoven/.local/lib/python3.8/site-packages/liegroups/torch/so3.py:71: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if ordering is 'xyzw':
/home/fantasticoven/.local/lib/python3.8/site-packages/liegroups/torch/so3.py:76: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif ordering is 'wxyz':
/home/fantasticoven/.local/lib/python3.8/site-packages/liegroups/torch/so3.py:362: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if ordering is 'xyzw':
/home/fantasticoven/.local/lib/python3.8/site-packages/liegroups/torch/so3.py:367: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif ordering is 'wxyz':
/home/fantasticoven/.local/lib/python3.8/site-packages/liegroups/torch/so3.py:71: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if ordering is 'xyzw':
/home/fantasticoven/.local/lib/python3.8/site-packages/liegroups/torch/so3.py:76: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif ordering is 'wxyz':
/home/fantasticoven/

IndexError: index 1 is out of bounds for dimension 0 with size 1

# Experiments for batch operations

In [ ]:
sigma = torch.ones(1, 3)
omiga = torch.normal(torch.zeros(1, 3), sigma)
omiga

In [ ]:
def transfer(omiga):
    omiga_0, omiga_1, omiga_2 = omiga[0], omiga[1], omiga[2]
    omiga_hat = torch.tensor([[0, -omiga_2, omiga_1],
                                [omiga_2, 0, -omiga_0],
                                [-omiga_1, omiga_0, 0]])
    return omiga_hat

In [ ]:
from functorch import vmap
batch_transfer = vmap(transfer)
batch_transfer(omiga)

In [ ]:
from liegroups.torch import SO3
C = SO3.exp(torch.Tensor([[1,2,3],
                          [0,0,0]]))
print(torch.Tensor([[1,2,3],
                          [0,0,0]]).size())
SO3.log(C)

In [ ]:
np.log(1)